In [ ]:
#remove #OTUID string (ONLY EXECUTE ONCE!)
!find output/table_out/*.txt -type f | while read FILENAME ; do sed -i.bak -E '1s/^.{8}//' "${FILENAME}"; done 
!rm output/table_out/*.bak

In [1]:
import numpy as np
import pandas as pd
from itertools import groupby
from os import listdir
from os.path import isfile, join

In [6]:
path='output/table_out/'
files = sorted([f for f in listdir(path) if isfile(join(path, f))])
sample_names = [files[i].split('_')[0][:-1] for i in np.arange(0,72,6)]
for i_sn in range(len(sample_names)):
    files_subset=files[6*i_sn:6*i_sn+6]
    df_arr=[]
    for f in files_subset:
        df_arr.append(pd.read_csv(path+f, delim_whitespace=True, header=0,index_col=0))
    print 'opened files: '+str(files_subset)
    print 'read '+str(len(df_arr))+' data frames from directory'
    #merge all dataframes together
    merge_df=pd.merge(df_arr[0],df_arr[1],left_index=True,right_index=True,how='outer')
    for i in range(2,len(df_arr)):
        merge_df=pd.merge(merge_df,df_arr[i],left_index=True,right_index=True,how='outer')
    merge_df=merge_df.fillna(0) #make all nans 0
    #count number of columns 
    col=0
    for data_frame in df_arr:
        col=col+data_frame.shape[1]
    print 'the number of columns summed from individual dataframes is '+str(col)
    print 'merged dataframe is '+str(merge_df.shape[1])+' columns and '+str(merge_df.shape[0])+' rows'
    
    #read in RDP taxonomy and assign to rdp_taxonomy field
    merge_df.index = [int(rec.split('Otu')[1]) for rec in merge_df.index.values]
    merge_df=merge_df.sort_index(axis=0)
    rdp_fix=np.loadtxt('output/allrank_otus.fa_classified.txt',dtype=str,delimiter='\n')
    rdp_fix=rdp_fix[6:]
    otu_index=[int(i.split(';',1)[0].split('Otu')[1]) for i in rdp_fix]
    rdp_fix=[i.split(';',1)[1].split(';',1)[1] for i in rdp_fix]
    rdp_all=np.loadtxt('output/fixrank_otus.fa_classified.txt',dtype=str,delimiter='\n')
    rdp_all=rdp_all[6:]
    rdp_all=[i.split(';',1)[1].split(';',1)[1] for i in rdp_all]

    #read in sequences
    def fasta_iter(fasta_name): #iterator for fasta files
        fh = open(fasta_name)
        faiter = (x[1] for x in groupby(fh, lambda line: line[0] == ">"))
        for header in faiter:
            header = header.next()[1:].strip()
            seq = "".join(s.strip() for s in faiter.next())
            yield header, seq
    seq=[]
    fiter=fasta_iter('output/otus.fa') #open fasta file via iterator
    for rec in fiter: #iterate through records
        h,s = rec #extract header and sequence
        seq.append(s) #extract sequence

    info_df = pd.DataFrame(index=otu_index)
    info_df['rdp_fix_taxonomy']=rdp_fix
    info_df['rdp_all_taxonomy']=rdp_all
    info_df['sequence']=seq
    merge_df=pd.merge(merge_df.astype(int),info_df,left_index=True,right_index=True,how='outer')
    merge_df=merge_df.fillna(0) #make all nans 0
    merge_df=merge_df.sort_index(axis=0)
    print 'output dataframe is '+str(merge_df.shape[1])+' columns and '+str(merge_df.shape[0])+' rows'
    merge_df.to_hdf('output/'+sample_names[i_sn]+'.h5','table',mode='w',complevel=9,complib='bzip2')
    print ''

opened files: ['FMTT11_97.txt', 'FMTT12_97.txt', 'FMTT13_97.txt', 'FMTT14_97.txt', 'FMTT15_97.txt', 'FMTT16_97.txt']
read 6 data frames from directory
the number of columns summed from individual dataframes is 53441
merged dataframe is 53441 columns and 3142 rows
output dataframe is 53444 columns and 5951 rows

opened files: ['FMTT21_97.txt', 'FMTT22_97.txt', 'FMTT23_97.txt', 'FMTT24_97.txt', 'FMTT25_97.txt', 'FMTT26_97.txt']
read 6 data frames from directory
the number of columns summed from individual dataframes is 51135
merged dataframe is 51135 columns and 3334 rows
output dataframe is 51138 columns and 5951 rows

opened files: ['FMTT31_97.txt', 'FMTT32_97.txt', 'FMTT33_97.txt', 'FMTT34_97.txt', 'FMTT35_97.txt', 'FMTT36_97.txt']
read 6 data frames from directory
the number of columns summed from individual dataframes is 37804
merged dataframe is 37804 columns and 3048 rows
output dataframe is 37807 columns and 5951 rows

opened files: ['FMTT41_97.txt', 'FMTT42_97.txt', 'FMTT43_97.t

In [ ]:
df=merge_df
plt.figure(figsize=(6, 10))
regex=['^FMTT11','^FMTT12','^FMTT13','^FMTT14','^FMTT15','^FMTT16']
cnt=1
for r in regex:
    plt.subplot(6,2,cnt)
    plt.hist(np.log10(df.filter(regex=r).sum(axis=0)), bins=25, color='k')
    plt.xlabel('log10(#reads)')
    plt.ylabel('#particles')
    plt.ylim([0,500])
    cnt+=1
    
    plt.subplot(6,2,cnt)
    fil=df.filter(regex=r)
    ordered=np.sort(np.array(fil.sum(axis=0)))[::-1]
    cum=np.zeros(ordered.shape)
    for i in range(ordered.shape[0]):
        cum[i]=ordered[i]
    plt.plot(range(cum.shape[0]), cum,'k-')
    plt.xlabel('particles, ordered largest to smallest')
    plt.ylabel('# reads per particle')
    plt.ylim([0,500])
    plt.xlim([0,1000])
    cnt+=1
plt.tight_layout()

In [ ]:
pres=df.filter(regex='FMTT16').T.loc[df.filter(regex='FMTT16').sum(axis=0)>100]

In [ ]:
import scipy.io, scipy.sparse
scipy.io.mmwrite("mmout", scipy.sparse.csr_matrix(merge_df))

In [ ]:
im_dad=scipy.sparse.csr_matrix(merge_df)